In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer

In [2]:
df=pd.read_excel('dataset_filledsupplier_currency_orderday.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24621 entries, 0 to 24620
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   청구서번호        24621 non-null  object 
 1   No.          24621 non-null  int64  
 2   Subject      24599 non-null  object 
 3   Machinery    24621 non-null  object 
 4   Assembly     24621 non-null  object 
 5   청구품목         24621 non-null  object 
 6   Unnamed: 6   0 non-null      float64
 7   Part No.1    24602 non-null  object 
 8   Part No.2    3592 non-null   object 
 9   청구량          24517 non-null  float64
 10  견적           24171 non-null  object 
 11  견적수량         24517 non-null  float64
 12  견적화폐         24621 non-null  object 
 13  견적단가         24621 non-null  float64
 14  발주번호         24621 non-null  object 
 15  발주처          24621 non-null  object 
 16  발주           24621 non-null  object 
 17  발주수량         24621 non-null  int64  
 18  발주금액         24621 non-null  float64
 19  D/T 

In [4]:
print(len(df['발주처'].unique()))

81


## 클리닝

In [5]:
missing_conditions = df[
    df['발주'].notnull() &  # 발주 일자는 비어있지 않음
    df['미입고 기간'].isnull() &  # 미입고 기간은 비어있음
    df['창고입고'].isnull() & # 창고 입고도 비어있음
    df['선박입고'].isnull()  # 선박 입고도 비어있음

]

print(f"발주 일자는 있지만 미입고 기간, 창고 입고, 선박 입고도 없는 경우: {len(missing_conditions)}개")
df = df.drop(missing_conditions.index)

print(f"삭제된 행의 개수: {len(missing_conditions)}개")
print(f"남은 데이터프레임의 크기: {df.shape}")

발주 일자는 있지만 미입고 기간, 창고 입고, 선박 입고도 없는 경우: 1699개
삭제된 행의 개수: 1699개
남은 데이터프레임의 크기: (22922, 32)


In [6]:
#미입고기간으로 처리.
missing_both = df[df['창고입고'].isnull() & df['미입고 기간'].notnull()]

print(f"창고 입고일은 없고 미입고 기간은 명시되어 있어 미입고 기간으로 분류해야 할 경우 : {len(missing_both)}개")

창고 입고일은 없고 미입고 기간은 명시되어 있어 미입고 기간으로 분류해야 할 경우 : 1620개


In [7]:
df = df[df['미입고 기간'].isnull()]

df['발주'] = pd.to_datetime(df['발주'], errors='coerce')
df['창고입고'] = pd.to_datetime(df['창고입고'], errors='coerce')

# 리드타임 계산
df['리드타임'] = (df['창고입고'] - df['발주']).dt.days
df['리드타임'] = df['리드타임'].apply(lambda x: 1 if x == 0 else x)

In [8]:

#df = df[(df['리드타임'] > 1 ) & (df['리드타임'] < 200)]
df = df.dropna(subset=['창고입고'])

In [9]:
print(df[['발주', '창고입고']].head(), df['창고입고'].isnull().sum())

          발주       창고입고
0 2019-01-11 2019-05-03
1 2019-01-11 2019-04-18
2 2019-01-11 2019-05-03
3 2019-04-01 2019-04-01
4 2019-04-01 2019-04-01 0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21302 entries, 0 to 24602
Data columns (total 33 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   청구서번호        21302 non-null  object        
 1   No.          21302 non-null  int64         
 2   Subject      21280 non-null  object        
 3   Machinery    21302 non-null  object        
 4   Assembly     21302 non-null  object        
 5   청구품목         21302 non-null  object        
 6   Unnamed: 6   0 non-null      float64       
 7   Part No.1    21286 non-null  object        
 8   Part No.2    3214 non-null   object        
 9   청구량          21198 non-null  float64       
 10  견적           20913 non-null  object        
 11  견적수량         21198 non-null  float64       
 12  견적화폐         21302 non-null  object        
 13  견적단가         21302 non-null  float64       
 14  발주번호         21302 non-null  object        
 15  발주처          21302 non-null  object        
 16  발주       

In [10]:
print(df.columns)
df['리드타임'] = df['리드타임'].astype(float)  # 필요시 데이터 타입 변환


Index(['청구서번호', 'No.', 'Subject', 'Machinery', 'Assembly', '청구품목',
       'Unnamed: 6', 'Part No.1', 'Part No.2', '청구량', '견적', '견적수량', '견적화폐',
       '견적단가', '발주번호', '발주처', '발주', '발주수량', '발주금액', 'D/T', '미입고 기간', '창고입고',
       '창고입고수량', 'Control No.', '입고창고', '창고출고', '창고출고수량', '출고선박', '출고운반선',
       '선박입고', '선박입고수량', '완료 여부', '리드타임'],
      dtype='object')


### 날짜뽑기

> 연도는 숫자형으로 처리(새로운 연도에 대응)
> 나머지는 원핫인코딩

In [10]:
# 월(month), 요일(day of the week) 추출
df['month'] = df['창고입고'].dt.month
df['day_of_week'] = df['창고입고'].dt.dayofweek
df['year'] = df['창고입고'].dt.year

def get_season(date):
    month = date.month
    if month in [3, 4, 5]:
        return '봄'
    elif month in [6, 7, 8]:
        return '여름'
    elif month in [9, 10, 11]:
        return '가을'
    else:
        return '겨울'

df['season'] = df['창고입고'].apply(get_season)

df = pd.get_dummies(df, columns=['month', 'day_of_week', 'season'], drop_first=True)


In [11]:
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'[^\w\s\*/\-\+.,#&]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\b(사용금지|사)\b', '', text, flags=re.IGNORECASE)
    text = text.strip()
    return text

def clean_supplier_name(name):
    name = name.lower()
    name = re.sub(r'coporation|coropration|coproration|corporration', 'corporation', name)
    name = re.sub(r'\(사용금지\)', '', name)
    name = re.sub(r'u\.s\.a', '_usa', name)
    name = re.sub(r'\.', '', name)
    suffixes = r'(corporation|corp|company|co|incorporated|inc|limited|ltd|상사|공사|엔지니어링|주식회사|주|gmbh|pte ltd|llc)'
    name = re.sub(suffixes, '', name, flags=re.IGNORECASE)
    name = re.sub(r'[^\w\s-]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name



In [12]:

text_columns = ['Machinery', 'Assembly', '청구품목']

for col in text_columns:
    df[col] = df[col].astype(str)
df['cleaned_machinery'] = df['Machinery'].apply(preprocess_text)
df['cleaned_assembly'] = df['Assembly'].apply(preprocess_text)
df['cleaned_item'] = df['청구품목'].apply(preprocess_text)
df['cleaned_supplier'] = df['발주처'].apply(clean_supplier_name)

In [13]:
# Machinery별 평균 리드타임 계산
machinery_avg_leadtime = df.groupby('cleaned_machinery')['리드타임'].mean().reset_index()
machinery_avg_leadtime.rename(columns={'리드타임': 'machinery_avg_leadtime'}, inplace=True)

# 데이터프레임에 평균 리드타임 추가
df = df.merge(machinery_avg_leadtime, on='cleaned_machinery', how='left')

In [14]:
# Machinery & Assembly 조합별 평균 리드타임 계산
df['machinery_assembly'] = df['cleaned_machinery'] + " & " + df['cleaned_assembly']
assembly_machinery_avg_leadtime = df.groupby('machinery_assembly')['리드타임'].mean().reset_index()
assembly_machinery_avg_leadtime.rename(columns={'리드타임': 'machinery_assembly_avg_leadtime'}, inplace=True)

# 데이터프레임에 Machinery & Assembly 조합의 평균 리드타임 추가
df = df.merge(assembly_machinery_avg_leadtime, on='machinery_assembly', how='left')


In [16]:
# 발주처별 평균 리드타임 계산 (cleaned_supplier 사용)
# supplier_leadtime_avg = df.groupby('cleaned_supplier')['리드타임'].mean().reset_index()
#supplier_leadtime_avg.rename(columns={'리드타임': 'supplier_avg_leadtime'}, inplace=True)

# 원본 데이터프레임에 발주처별 평균 리드타임 추가 (cleaned_supplier로 병합)
# df = df.merge(supplier_leadtime_avg, on='cleaned_supplier', how='left')


In [15]:
# 발주처별 + Machinery 조합별 평균 리드타임 계산
supplier_machinery_leadtime = df.groupby(['cleaned_supplier', 'cleaned_machinery'])['리드타임'].mean().reset_index()
supplier_machinery_leadtime.rename(columns={'리드타임': 'supplier_machinery_avg_leadtime'}, inplace=True)

# 원본 데이터프레임에 병합
df = df.merge(supplier_machinery_leadtime, on=['cleaned_supplier', 'cleaned_machinery'], how='left')

In [16]:
df[['Machinery','Assembly','리드타임', '발주처', 'machinery_avg_leadtime','machinery_assembly_avg_leadtime', 'supplier_machinery_avg_leadtime']].head(20)

,Machinery,Assembly,리드타임,발주처,machinery_avg_leadtime,machinery_assembly_avg_leadtime,supplier_machinery_avg_leadtime
0,CARGO BOOM VANG BLOCK (STBD 하),BLOCK,112,MATSUI(U.S.A) COROPRATION,98.333333,102.000000,98.333333
1,SPANISH BOOM VANG BLOCK (PORT 상),BLOCK,97,MATSUI(U.S.A) COROPRATION,72.000000,63.750000,121.000000
2,PURSE BLOCK,TOW BLOCK,112,MATSUI(U.S.A) COROPRATION,100.000000,91.333333,97.666667
3,NET,H-EX,12,KTI,16.627119,16.642857,16.675926
4,NET,NYLON,12,KTI,16.627119,12.190476,16.675926
5,NET,NYLON,12,KTI,16.627119,12.190476,16.675926
6,NET,NYLON,12,KTI,16.627119,12.190476,16.675926
7,NET,NYLON,12,KTI,16.627119,12.190476,16.675926
8,NET,NYLON,12,KTI,16.627119,12.190476,16.675926
9,NET,H-EX,12,KTI,16.627119,16.642857,16.675926


In [10]:
def remove_restricted_text(text):
    return re.sub(r'\(사용금지\)', '', text).strip()

# Remove "(사용금지)" from '청구품목' and '발주처' columnsdf['청구품목'] = df['청구품목'].apply(remove_restricted_text)
df['발주처'] = df['발주처'].apply(remove_restricted_text)

In [18]:
fixed_columns = [
    'cleaned_machinery', 'cleaned_assembly', '청구품목',  'cleaned_supplier', '리드타임',
    'machinery_avg_leadtime', 'machinery_assembly_avg_leadtime', 'supplier_machinery_avg_leadtime', '견적화폐'
]

# 해당 컬럼들만 선택하여 새로운 데이터프레임 생성
df_fixed = df[fixed_columns]

# 고정 피처들을 CSV로 저장
df_fixed.to_csv('leadtime_features.csv', index=False, quotechar='"', quoting=csv.QUOTE_MINIMAL)

print("고정 피처가 포함된 CSV 파일이 저장되었습니다.")

고정 피처가 포함된 CSV 파일이 저장되었습니다.


In [19]:
past_columns = [
    'Machinery', 'Assembly', '청구품목', 'Part No.1', '견적화폐', '견적단가', '발주처', '발주', '창고입고', '리드타임'
]

# 해당 컬럼들만 선택하여 새로운 데이터프레임 생성
df_past = df[past_columns]

# 고정 피처들을 CSV로 저장
df_past.to_csv('past_leadtime.csv', index=False, quotechar='"', quoting=csv.QUOTE_MINIMAL)

print("과거 리드타임 포함된 CSV 파일이 저장되었습니다.")

과거 리드타임 포함된 CSV 파일이 저장되었습니다.


In [27]:
# Sample data를 DataFrame으로 불러오기
data = pd.read_csv('past_leadtime.csv', quoting=csv.QUOTE_MINIMAL)

# 그룹화하여 '발주'와 '창고입고'가 다른 경우 찾기
group_columns = ['Machinery', 'Assembly', '청구품목', 'Part No.1', '견적화폐', '견적단가', '발주처']
unique_groups = data.groupby(group_columns)

# 동일한 그룹 내에서 '발주' 또는 '창고입고'가 다른 경우 필터링
different_lead_time = unique_groups.filter(lambda x: len(x[['발주', '창고입고']].drop_duplicates()) > 1)

# 다시 동일한 '발주'와 '창고입고'가 동일한 경우 제거
different_lead_time = different_lead_time.drop_duplicates(subset=group_columns + ['발주', '창고입고'], keep=False)

# 그룹화된 열들을 기준으로 정렬하여 동일한 항목이 인접하도록 함
different_lead_time_sorted = different_lead_time.sort_values(by=group_columns + ['발주', '창고입고'])

# 결과 확인
print(different_lead_time_sorted.head(40))

                Machinery                        Assembly  \
711           6M NET BOAT                       GENERAOTR   
8980          6M NET BOAT                       GENERATOR   
11853         6M NET BOAT                       GENERATOR   
8299          6M NET BOAT                       GENERATOR   
10825         6M NET BOAT                       GENERATOR   
528           6M NET BOAT            NET BOAT FISH FINDER   
18461         6M NET BOAT            NET BOAT FISH FINDER   
6314          6M NET BOAT                 VULKAN COUPLING   
1494          6M NET BOAT                 VULKAN COUPLING   
13838  6M NET BOAT ENGINE       198-4112 MOTOR GP-ELEC -C   
14586  6M NET BOAT ENGINE       198-4112 MOTOR GP-ELEC -C   
97     6M NET BOAT ENGINE              ENGINE STOP DEVICE   
12892  6M NET BOAT ENGINE              ENGINE STOP DEVICE   
3526   6M NET BOAT ENGINE           FIG 1. CYLINDER BLOCK   
11410  6M NET BOAT ENGINE           FIG 1. CYLINDER BLOCK   
15423  6M NET BOAT ENGIN

In [25]:
unique_group_count = different_lead_time[group_columns].drop_duplicates().shape[0]

# 결과 출력
print(f"서로 다른 '발주'와 '창고입고'를 가진 고유한 그룹의 개수: {unique_group_count}")

서로 다른 '발주'와 '창고입고'를 가진 고유한 그룹의 개수: 2357
